In [9]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
from geopandas import GeoDataFrame
from shapely.geometry import shape
from geopandas import read_file
import json
from shapely.geometry import shape, Point, Polygon
from geopy.distance import geodesic
import requests
import time
import pprint
import numpy as np


pp = pprint.PrettyPrinter(indent=1)


In [15]:
# Load the CSV file
file_path = './Recreation_and_Parks_Properties.csv'
parks = pd.read_csv(file_path)

#read the first few pieces of data
parks.head()

,Map_Label,Longitude,Latitude,Acres,TMA_PropertyID,SquareFeet,PerimeterLength,PropertyType,Address,City,...,State_Senate,MONS_Neighborhood,Police_District,US_Congress,Realtor_Neighborhood,State_Assembly,Planning_Neighborhood,shape,created_date,last_edited_date
0,Maritime Plaza,-122.399239,37.795558,2.013068,142,8.767600e+04,3208.815408,Civic Plaza or Square,285 Washington Street,San Francisco,...,11th Senatorial District,Financial District,Central,12th Congressional District,Financial District/Barbary Coast,17th Assembly District,Financial District,"MULTIPOLYGON (((-122.3993869 37.7952566, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
1,Lake Merced Park,-122.492377,37.721835,608.578437,10,2.650565e+07,33386.362623,Regional Park,Lake Merced Boulevard,San Francisco,...,11th Senatorial District,Lakeshore,Taraval,14th Congressional District,Lake Shore,19th Assembly District,Lakeshore,"MULTIPOLYGON (((-122.4961066 37.7289285, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
2,Broadway Tunnel West Mini Park,-122.417423,37.796436,0.124525,203,5.423487e+03,317.608457,Mini Park,1201 Broadway,San Francisco,...,11th Senatorial District,"Russian Hill,Nob Hill",Central,12th Congressional District,Nob Hill,17th Assembly District,Nob Hill,"MULTIPOLYGON (((-122.4172362 37.796398, -122.4...",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
3,Mt. Davidson Park,-122.454654,37.739063,40.717980,69,1.773406e+06,7160.937556,Regional Park,Myra Way,San Francisco,...,11th Senatorial District,"Sherwood Forest,Miraloma Park",Ingleside,12th Congressional District,Miraloma Park,17th Assembly District,West of Twin Peaks,"MULTIPOLYGON (((-122.4530944 37.7409793, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
4,Union Square,-122.407500,37.787931,2.600105,135,1.132433e+05,1373.590639,Civic Plaza or Square,Post & Stockton,San Francisco,...,11th Senatorial District,Downtown / Union Square,Central,12th Congressional District,Downtown,17th Assembly District,Downtown/Civic Center,"MULTIPOLYGON (((-122.4067349 37.7876523, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM


In [16]:
#filter data to only include properties whose property types are "Neighborhood Park or Playground", "Regional Park", "Civic Plaza or Square", or "Mini Park"
filtered_parks = parks[parks['PropertyType'].isin(['Neighborhood Park or Playground', 'Regional Park', 'Civic Plaza or Square', 'Mini Park'])]
filtered_parks.head()

,Map_Label,Longitude,Latitude,Acres,TMA_PropertyID,SquareFeet,PerimeterLength,PropertyType,Address,City,...,State_Senate,MONS_Neighborhood,Police_District,US_Congress,Realtor_Neighborhood,State_Assembly,Planning_Neighborhood,shape,created_date,last_edited_date
0,Maritime Plaza,-122.399239,37.795558,2.013068,142,8.767600e+04,3208.815408,Civic Plaza or Square,285 Washington Street,San Francisco,...,11th Senatorial District,Financial District,Central,12th Congressional District,Financial District/Barbary Coast,17th Assembly District,Financial District,"MULTIPOLYGON (((-122.3993869 37.7952566, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
1,Lake Merced Park,-122.492377,37.721835,608.578437,10,2.650565e+07,33386.362623,Regional Park,Lake Merced Boulevard,San Francisco,...,11th Senatorial District,Lakeshore,Taraval,14th Congressional District,Lake Shore,19th Assembly District,Lakeshore,"MULTIPOLYGON (((-122.4961066 37.7289285, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
2,Broadway Tunnel West Mini Park,-122.417423,37.796436,0.124525,203,5.423487e+03,317.608457,Mini Park,1201 Broadway,San Francisco,...,11th Senatorial District,"Russian Hill,Nob Hill",Central,12th Congressional District,Nob Hill,17th Assembly District,Nob Hill,"MULTIPOLYGON (((-122.4172362 37.796398, -122.4...",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
3,Mt. Davidson Park,-122.454654,37.739063,40.717980,69,1.773406e+06,7160.937556,Regional Park,Myra Way,San Francisco,...,11th Senatorial District,"Sherwood Forest,Miraloma Park",Ingleside,12th Congressional District,Miraloma Park,17th Assembly District,West of Twin Peaks,"MULTIPOLYGON (((-122.4530944 37.7409793, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
4,Union Square,-122.407500,37.787931,2.600105,135,1.132433e+05,1373.590639,Civic Plaza or Square,Post & Stockton,San Francisco,...,11th Senatorial District,Downtown / Union Square,Central,12th Congressional District,Downtown,17th Assembly District,Downtown/Civic Center,"MULTIPOLYGON (((-122.4067349 37.7876523, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM


In [20]:
#remove all entries whose map_label contains "playground"
parks_without_playgrounds = filtered_parks[~filtered_parks['Map_Label'].str.contains('playground', case=False)]
parks_without_playgrounds.head()

,Map_Label,Longitude,Latitude,Acres,TMA_PropertyID,SquareFeet,PerimeterLength,PropertyType,Address,City,...,State_Senate,MONS_Neighborhood,Police_District,US_Congress,Realtor_Neighborhood,State_Assembly,Planning_Neighborhood,shape,created_date,last_edited_date
0,Maritime Plaza,-122.399239,37.795558,2.013068,142,8.767600e+04,3208.815408,Civic Plaza or Square,285 Washington Street,San Francisco,...,11th Senatorial District,Financial District,Central,12th Congressional District,Financial District/Barbary Coast,17th Assembly District,Financial District,"MULTIPOLYGON (((-122.3993869 37.7952566, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
1,Lake Merced Park,-122.492377,37.721835,608.578437,10,2.650565e+07,33386.362623,Regional Park,Lake Merced Boulevard,San Francisco,...,11th Senatorial District,Lakeshore,Taraval,14th Congressional District,Lake Shore,19th Assembly District,Lakeshore,"MULTIPOLYGON (((-122.4961066 37.7289285, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
2,Broadway Tunnel West Mini Park,-122.417423,37.796436,0.124525,203,5.423487e+03,317.608457,Mini Park,1201 Broadway,San Francisco,...,11th Senatorial District,"Russian Hill,Nob Hill",Central,12th Congressional District,Nob Hill,17th Assembly District,Nob Hill,"MULTIPOLYGON (((-122.4172362 37.796398, -122.4...",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
3,Mt. Davidson Park,-122.454654,37.739063,40.717980,69,1.773406e+06,7160.937556,Regional Park,Myra Way,San Francisco,...,11th Senatorial District,"Sherwood Forest,Miraloma Park",Ingleside,12th Congressional District,Miraloma Park,17th Assembly District,West of Twin Peaks,"MULTIPOLYGON (((-122.4530944 37.7409793, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM
4,Union Square,-122.407500,37.787931,2.600105,135,1.132433e+05,1373.590639,Civic Plaza or Square,Post & Stockton,San Francisco,...,11th Senatorial District,Downtown / Union Square,Central,12th Congressional District,Downtown,17th Assembly District,Downtown/Civic Center,"MULTIPOLYGON (((-122.4067349 37.7876523, -122....",06/26/2023 07:11:19 PM,06/26/2023 07:11:19 PM


In [21]:
#convert to geojson


gdf = gpd.GeoDataFrame(parks_without_playgrounds, geometry=gpd.points_from_xy(parks_without_playgrounds.Longitude, parks_.Latitude))



In [22]:
# Save as a GeoJSON file
geojson_file_path_gpd = "./parks.geojson"
gdf.to_file(geojson_file_path_gpd, driver="GeoJSON")


# PROXIMITY TO PARKS

In [23]:
#WITH THIS NEW GEOJSON FILE, WE CAN NOW START





In [24]:
#OPEN FILES
with open('./neighborhoods.geojson', 'r') as file:
    neighborhoods_data = json.load(file)
with open('./parks.geojson', 'r') as file:
    parks_data = json.load(file)

In [42]:
def min_distance_to_park(neighborhood, parks):
    # Handling the multi-polygon structure
    neighborhood_coords = neighborhood['geometry']['coordinates'][0][0]
    neighborhood_polygon = Polygon(neighborhood_coords)
    centroid_coords = (neighborhood_polygon.centroid.y, neighborhood_polygon.centroid.x) # Swapping latitude and longitude
    
    min_distance = float('inf')
    for park in parks['features']:
        park_coords = (park['geometry']['coordinates'][1], park['geometry']['coordinates'][0]) # Making sure the coordinates are in the correct format
        distance = geodesic(centroid_coords, park_coords).meters
        min_distance = min(min_distance, distance)
        #returning centroid_coords as well so we can use this later
    return min_distance

# Calculating the nearest park distances for all neighborhoods
nearest_park_distances = [(min_distance_to_park(neighborhood, parks_data)) for neighborhood in neighborhoods_data['features']]

neighborhoods_min_distance_dict={}

for neighborhood in neighborhoods_data['features']:
#     print (neighborhood)
    neighborhoods_min_distance_dict[neighborhood['properties']['name']]=min_distance_to_park(neighborhood, parks_data)

                                                             

# Sorting the distances
sorted_neighborhoods_min_distance_dict = sorted(neighborhoods_min_distance_dict.items(), key=lambda x:x[1], reverse=False)

# Finding the values at the 25th and 75th percentiles
q1_index = len(sorted_neighborhoods_min_distance_dict) // 4
q2_index = 2 * len(sorted_neighborhoods_min_distance_dict) // 4
q3_index= 3 * len(sorted_neighborhoods_min_distance_dict) // 4

#now, we have the indices for the first and third quartiles. everything below first gets a score of 3, 2, 1, 0.
# Assigning scores and updating the neighborhoods data
counter = 0
for neighborhood_name, distance in sorted_neighborhoods_min_distance_dict:
    if counter < q1_index:
        score = 3
    elif counter < q2_index:
        score = 2
    elif counter < q3_index:
        score = 1
    else:
        score = 0

    # Finding the corresponding feature and adding the score property
    for feature in neighborhoods_data['features']:
        if feature['properties']['name'] == neighborhood_name:
            feature['properties']['park_score'] = score
            break

    counter += 1
    
# Saving the modified GeoJSON object
output_path = './neighborhoods_with_scores.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)



In [26]:
with open('./neighborhoods_with_scores.geojson', 'r') as file:
    updated_neighborhoods_data = json.load(file)
updated_neighborhoods_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'link': 'http://en.wikipedia.org/wiki/Sea_Cliff,_San_Francisco,_California',
    'name': 'Seacliff',
    'nearest_park_distance': 0.0009406020243021834,
    'park_score': 1},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-122.49345526799993, 37.78351817100008],
       [-122.49372649999992, 37.78724665100009],
       [-122.49358666699993, 37.78731259500006],
       [-122.49360569399994, 37.78752774600008],
       [-122.49283007399993, 37.787882585000034],
       [-122.4927566799999, 37.78773917700005],
       [-122.48982906399993, 37.789482184000065],
       [-122.48899105699991, 37.78928318700008],
       [-122.4878640209999, 37.78958817900008],
       [-122.48736904899994, 37.78942984100007],
       [-122.48598032899991, 37.79080370600008],
       [-122.48581537399991, 37.79070384600004],
       [-122.48557750799989, 37.790559847000054],
       [-122.4850531269999, 37.79036813300007],
  

# UPDATING QUADRANTS OF EACH NEIGHBORHOOD

In [28]:
# Function to convert GeoJSON to GeoDataFrame
def geojson_to_geodataframe(geojson):
    features = geojson['features']
    geometries = [shape(feature['geometry']) for feature in features]
    properties = [feature['properties'] for feature in features]
    return GeoDataFrame(properties, geometry=geometries)

# Loading neighborhoods and quadrants GeoJSON files
with open('./neighborhoods.geojson', 'r') as file:
    neighborhoods_geojson = json.load(file)
with open('./quadrants.geojson', 'r') as file:
    quadrants_geojson = json.load(file)

# Converting to GeoDataFrames
neighborhoods_gdf = geojson_to_geodataframe(neighborhoods_geojson)
quadrants_gdf = geojson_to_geodataframe(quadrants_geojson)

# Performing spatial join to find the overlapping quadrants for each neighborhood
joined_gdf = GeoDataFrame(neighborhoods_gdf)
joined_gdf['quad'] = None  # Initialize the 'quad' column

for idx, neighborhood in neighborhoods_gdf.iterrows():
    for _, quadrant in quadrants_gdf.iterrows():
        if neighborhood['geometry'].intersects(quadrant['geometry']):
            joined_gdf.at[idx, 'quad'] = quadrant['quad']
            break

# Converting back to GeoJSON
joined_geojson = json.loads(joined_gdf.to_json())

# Saving the updated GeoJSON file with the "quad" property added
updated_file_path = './updated_neighborhoods.geojson'
with open(updated_file_path, 'w') as file:
    json.dump(joined_geojson, file)

updated_file_path

'./updated_neighborhoods.geojson'

# DISPLAYING CRIME DATA FOR EACH NEIGHBORHOOD BASED ON GEOJSON FILE

## FILTER THE DATA

In [29]:
# Load the CSV file
file_path = './all_crimes.csv'
crimes = pd.read_csv(file_path)

#read the first few pieces of data
crimes.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#filter data to only include properties whose property types are "Neighborhood Park or Playground", "Regional Park", "Civic Plaza or Square", or "Mini Park"
filtered_crimes = crimes[crimes['Incident Year'].isin([2023])]

filtered_crimes.head()

                   

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Filtering the crimes that have valid values for both Latitude and Longitude columns
filtered_crimes_data = filtered_crimes.dropna(subset=['Longitude', 'Latitude'])


In [32]:
#convert to geojson

gdf = gpd.GeoDataFrame(filtered_crimes_data, geometry=gpd.points_from_xy(filtered_crimes_data.Longitude, filtered_crimes_data.Latitude))

# Save as a GeoJSON file
geojson_file_path_gpd = "./all_filtered_crimes_data.geojson"
gdf.to_file(geojson_file_path_gpd, driver="GeoJSON")


## USE A SPATIAL JOIN TO CALCULATE NUMBER OF CRIMES PER NEIGHBORHOOD

In [33]:

# Loading the neighborhoods data
neighborhoods_file_path = "./updated_neighborhoods.geojson"
neighborhoods_data = gpd.read_file(neighborhoods_file_path)

# Loading the crimes data
crimes_file_path = "./all_filtered_crimes_data.geojson"
crimes_data = gpd.read_file(crimes_file_path)



In [34]:
# Perform a spatial join to find crimes within the neighborhood regions
crimes_within_neighborhoods = gpd.sjoin(crimes_data, neighborhoods_data, predicate='within')

# Group by neighborhood name and count the number of crimes for each
number_of_crimes = crimes_within_neighborhoods.groupby('name').size().reset_index(name='number_of_crimes')

# Merge the counts back to the neighborhoods data
neighborhoods_with_crimes = neighborhoods_data.merge(number_of_crimes, on='name', how='left')

# Fill any missing values with 0 (for neighborhoods with no crimes)
neighborhoods_with_crimes['number_of_crimes'] = neighborhoods_with_crimes['number_of_crimes'].fillna(0).astype(int)

# Export the updated GeoJSON file
output_file_path = "neighborhoods_with_crimes.geojson"
neighborhoods_with_crimes.to_file(output_file_path, driver='GeoJSON')

print(f"File exported to {output_file_path}")

File exported to neighborhoods_with_crimes.geojson


## ADD A COLUMN FOR SAFETY SCORE OUT OF 4

In [35]:

updated_neighborhoods_file_path = "./neighborhoods_with_crimes.geojson"
neighborhoods_with_crimes_updated = gpd.read_file(updated_neighborhoods_file_path)

# Selecting the "name" and "number_of_crimes" columns to display the crime counts for each neighborhood
crime_counts_updated = neighborhoods_with_crimes_updated[['name', 'number_of_crimes']]


# Selecting the "name" and "number_of_crimes" columns to display the crime counts for each neighborhood
crime_counts_updated = neighborhoods_with_crimes_updated[['name', 'number_of_crimes']]


# Calculating the quartiles for the "number_of_crimes" column
quartiles = neighborhoods_with_crimes_updated['number_of_crimes'].quantile([0.25, 0.5, 0.75])


# Defining a function to assign a crime score based on quartiles
def assign_crime_score(number_of_crimes):
    if number_of_crimes >= quartiles[0.75]:
        return 0
    elif number_of_crimes >= quartiles[0.5]:
        return 1
    elif number_of_crimes >= quartiles[0.25]:
        return 2
    else:
        return 3

# Applying the function to the "number_of_crimes" column to create the "crime_score" column
neighborhoods_with_crimes_updated['crime_score'] = neighborhoods_with_crimes_updated['number_of_crimes'].apply(assign_crime_score)

# Displaying the updated data with the new "crime_score" column
neighborhoods_with_crimes_updated[['name', 'number_of_crimes', 'crime_score']]

# Define the output file path for the updated GeoJSON file
output_updated_file_path = "./neighborhoods_with_crimes_and_score.geojson"

# Exporting the updated neighborhoods data with crime scores to a new GeoJSON file
neighborhoods_with_crimes_updated.to_file(output_updated_file_path, driver='GeoJSON')

# Confirming the file creation
output_updated_file_path

'./neighborhoods_with_crimes_and_score.geojson'

# GETTING ALL BIKE PATHS

In [46]:
stations = requests.get("https://gbfs.lyft.com/gbfs/1.1/bay/en/station_information.json")

station_names_and_coords={}
for station in stations.json()["data"]["stations"]:
    print (station)
    station_names_and_coords[station["name"]]=[station["lon"],station["lat"]]
    
for station in station_names_and_coords:
    print (station, station_names_and_coords[station])

{'name': 'Newbury Park Dr at King Rd', 'external_id': '6994e1e6-bb3c-4309-8207-52e6004a7302', 'has_kiosk': True, 'capacity': 23, 'eightd_has_key_dispenser': False, 'short_name': 'SJ-H17', 'lon': -121.86796635389327, 'lat': 37.3655356585599, 'rental_uris': {'ios': 'https://sfo.lft.to/lastmile_qr_scan', 'android': 'https://sfo.lft.to/lastmile_qr_scan'}, 'station_type': 'classic', 'rental_methods': ['KEY', 'CREDITCARD'], 'region_id': '5', 'station_id': '6994e1e6-bb3c-4309-8207-52e6004a7302', 'eightd_station_services': [], 'electric_bike_surcharge_waiver': False}
{'name': 'Bestor Art Park', 'external_id': '46b4ef45-b06b-40eb-9fdf-9bc8ff104a4f', 'has_kiosk': True, 'capacity': 15, 'eightd_has_key_dispenser': False, 'short_name': 'SJ-Q11', 'lon': -121.8741186, 'lat': 37.3236779, 'rental_uris': {'ios': 'https://sfo.lft.to/lastmile_qr_scan', 'android': 'https://sfo.lft.to/lastmile_qr_scan'}, 'station_type': 'classic', 'rental_methods': ['KEY', 'CREDITCARD'], 'region_id': '5', 'station_id': '46b

## SCORING EACH NEIGHBORHOOD BASED ON DISTANCE TO NEAREST BIKE PATHS LIKE PARKS

In [40]:
#Getting last updated neighborhoods_data file
with open('./neighborhoods_with_crimes_and_score.geojson', 'r') as file:
    neighborhoods_data = json.load(file)
neighborhoods_data

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'id': '0',
    'link': 'http://en.wikipedia.org/wiki/Sea_Cliff,_San_Francisco,_California',
    'name': 'Seacliff',
    'nearest_park_distance': 0.0009406020243021834,
    'park_score': 1,
    'quad': 'NW',
    'number_of_crimes': 70,
    'crime_score': 3},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-122.49345526799993, 37.78351817100008],
       [-122.49372649999992, 37.78724665100009],
       [-122.49358666699993, 37.78731259500006],
       [-122.49360569399994, 37.78752774600008],
       [-122.49283007399993, 37.787882585000034],
       [-122.4927566799999, 37.78773917700005],
       [-122.48982906399993, 37.789482184000065],
       [-122.48899105699991, 37.78928318700008],
       [-122.4878640209999, 37.78958817900008],
       [-122.48736904899994, 37.78942984100007],
       [-122.48598032899991, 3

In [47]:
def min_distance_to_bike_station(neighborhood, stations):
    # Handling the multi-polygon structure
    neighborhood_coords = neighborhood['geometry']['coordinates'][0][0]
    neighborhood_polygon = Polygon(neighborhood_coords)
    centroid_coords = (neighborhood_polygon.centroid.y, neighborhood_polygon.centroid.x) # Swapping latitude and longitude
    
    min_distance = float('inf')
    for station in stations:
        station_coords = (stations[station][1], stations[station][0]) # Extracting latitude and longitude
        distance = geodesic(centroid_coords, station_coords).meters
        min_distance = min(min_distance, distance)
        # returning centroid_coords as well so we can use this later
    return min_distance

# Calculating the nearest park distances for all neighborhoods
nearest_station_distances = [(min_distance_to_bike_station(neighborhood, station_names_and_coords)) for neighborhood in neighborhoods_data['features']]

neighborhoods_min_distance_dict={}

for neighborhood in neighborhoods_data['features']:
#     print (neighborhood)
    neighborhoods_min_distance_dict[neighborhood['properties']['name']]=min_distance_to_bike_station(neighborhood, station_names_and_coords)

                                                             

# Sorting the distances
sorted_neighborhoods_min_distance_dict = sorted(neighborhoods_min_distance_dict.items(), key=lambda x:x[1], reverse=False)

# Finding the values at the 25th and 75th percentiles
q1_index = len(sorted_neighborhoods_min_distance_dict) // 4
q2_index = 2 * len(sorted_neighborhoods_min_distance_dict) // 4
q3_index= 3 * len(sorted_neighborhoods_min_distance_dict) // 4

#now, we have the indices for the first and third quartiles. everything below first gets a score of 3, 2, 1, 0.
# Assigning scores and updating the neighborhoods data
counter = 0
for neighborhood_name, distance in sorted_neighborhoods_min_distance_dict:
    if counter > q3_index:
        score = 3
    elif counter > q2_index:
        score = 2
    elif counter > q1_index:
        score = 1
    else:
        score = 0

    # Finding the corresponding feature and adding the score property
    for feature in neighborhoods_data['features']:
        if feature['properties']['name'] == neighborhood_name:
            feature['properties']['bike_score'] = score
            break

    counter += 1
    
# Saving the modified GeoJSON object
output_path = './neighborhoods_with_bike_scores.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)



# GETTING PUBLIC TRANSIT DATA

In [75]:
operator_response=requests.get(" http://api.511.org/transit/operators?api_key=${apiKey}")


# Decode the content using 'utf-8-sig' to remove the BOM
operator_json = operator_response.content.decode('utf-8-sig')

operator_data = json.loads(transit_json)
operator_ids=[]
for operator in operator_data:
    print(operator)
    operator_ids.append(operator["Id"])

#Run API calls to get every stop with every operator id 

{'Id': '840', 'Name': 'Foothill - Eastmont All Nighter', 'TransportMode': 'bus', 'PublicCode': '840', 'SiriLineRef': '840', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '95', 'Name': 'D St. - Maud - Fairview', 'TransportMode': 'bus', 'PublicCode': '95', 'SiriLineRef': '95', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '51A', 'Name': 'Broadway - Santa Clara', 'TransportMode': 'bus', 'PublicCode': '51A', 'SiriLineRef': '51A', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '251', 'Name': 'Paseo Padre - Thornton - Cherry', 'TransportMode': 'bus', 'PublicCode': '251', 'SiriLineRef': '251', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '51B', 'Name': 'University - College - Rockridge', 'TransportMode': 'bus', 'PublicCode': '51B', 'SiriLineRef': '51B', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '54', 'Name': '35th Ave. - Merritt College', 'TransportMode': 'bus', 'PublicCode': '54', 'SiriLineRef': '54', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '93', 'Name': 'Ashland - San Lorenz

In [74]:
operator_response=requests.get(" http://api.511.org/transit/lines?api_key={apiKey}")


# Decode the content using 'utf-8-sig' to remove the BOM
operator_json = operator_response.content.decode('utf-8-sig')

operator_data = json.loads(transit_json)
operator_ids=[]
for operator in operator_data:
    print(operator)
    operator_ids.append(operator["Id"])

#Run API calls to get every stop with every operator id 

{'Id': '840', 'Name': 'Foothill - Eastmont All Nighter', 'TransportMode': 'bus', 'PublicCode': '840', 'SiriLineRef': '840', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '95', 'Name': 'D St. - Maud - Fairview', 'TransportMode': 'bus', 'PublicCode': '95', 'SiriLineRef': '95', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '51A', 'Name': 'Broadway - Santa Clara', 'TransportMode': 'bus', 'PublicCode': '51A', 'SiriLineRef': '51A', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '251', 'Name': 'Paseo Padre - Thornton - Cherry', 'TransportMode': 'bus', 'PublicCode': '251', 'SiriLineRef': '251', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '51B', 'Name': 'University - College - Rockridge', 'TransportMode': 'bus', 'PublicCode': '51B', 'SiriLineRef': '51B', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '54', 'Name': '35th Ave. - Merritt College', 'TransportMode': 'bus', 'PublicCode': '54', 'SiriLineRef': '54', 'Monitored': True, 'OperatorRef': 'AC'}
{'Id': '93', 'Name': 'Ashland - San Lorenz

# GETTING BART DATA

In [89]:
# do this with an api later, still waiting on a key though. For now, manually plug it in.
bart_stations = {
    "Embarcadero Station": [37.792976, -122.396742],
    "Montgomery St. Station": [37.789336, -122.401486],
    "Powell St. Station": [37.784991, -122.406857],
    "Civic Center/UN Plaza Station": [37.779528, -122.413756],
    "16th St. Mission Station": [37.764854, -122.419974],
    "24th St. Mission Station": [37.752396, -122.418188],
    "Glen Park Station": [37.733104, -122.433817],
    "Balboa Park Station": [37.721585, -122.447506],
    "Colma Station": [37.684638, -122.466233]
}

## Find the nearest walking distance and the nearest driving distance from every neighborhood to every bart station

In [330]:
with open('./neighborhoods_with_bike_scores.geojson', 'r') as file:
    neighborhoods_data = json.load(file)

In [334]:


def get_route_summary(origin_lat, origin_lon, dest_lat, dest_lon, travel_mode, bing_maps_key):
    # Form the URL for Bing Maps Routes API
    url = f"http://dev.virtualearth.net/REST/v1/Routes/{travel_mode}?wayPoint.1={origin_lat},{origin_lon}&wayPoint.2={dest_lat},{dest_lon}&distanceUnit=mi&key={bing_maps_key}"
    response = requests.get(url)  
    if response.status_code == 200:
        route_data = response.json()
        # Get the travel duration in seconds
        route_duration = route_data['resourceSets'][0]['resources'][0]['travelDuration']
        return route_duration
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None
    
#function for getting minimum driving distance from a neighborhood to each station
def min_driving_distance_to_bart(neighborhood, stations):


    # Handling the multi-polygon structure
    neighborhood_coords = neighborhood['geometry']['coordinates'][0][0]
    neighborhood_polygon = Polygon(neighborhood_coords)
    centroid_coords = (neighborhood_polygon.centroid.y, neighborhood_polygon.centroid.x) # Swapping latitude and longitude
    
    min_distance = float('inf')
    for station_name, station_coords in stations.items():
        distance = get_route_summary(centroid_coords[0], centroid_coords[1], station_coords[0], station_coords[1], "Driving", api_key)
        if distance:
            min_distance = min(min_distance, distance)

    return min_distance


In [338]:
print (neighborhoods_data['features'])
# Assuming neighborhoods_data and bart_stations are defined
neighborhoods_min_distance_dict = {}
for neighborhood in neighborhoods_data['features']:
    neighborhood_name = neighborhood['properties']['name']
    min_distance = min_driving_distance_to_bart(neighborhood, bart_stations)
    print (min_distance)
    neighborhoods_min_distance_dict[neighborhood_name] = min_distance

print(neighborhoods_min_distance_dict)
# Sorting the distances
sorted_neighborhoods_min_distance = sorted(neighborhoods_min_distance_dict.items(), key=lambda x:x[1])




[{'type': 'Feature', 'properties': {'id': '0', 'link': 'http://en.wikipedia.org/wiki/Sea_Cliff,_San_Francisco,_California', 'name': 'Seacliff', 'nearest_park_distance': 0.0009406020243021834, 'park_score': 1, 'quad': 'NW', 'number_of_crimes': 70, 'crime_score': 3, 'bike_score': 2, 'driving_bart_score': 0}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-122.49345526799993, 37.78351817100008], [-122.49372649999992, 37.78724665100009], [-122.49358666699993, 37.78731259500006], [-122.49360569399994, 37.78752774600008], [-122.49283007399993, 37.787882585000034], [-122.4927566799999, 37.78773917700005], [-122.48982906399993, 37.789482184000065], [-122.48899105699991, 37.78928318700008], [-122.4878640209999, 37.78958817900008], [-122.48736904899994, 37.78942984100007], [-122.48598032899991, 37.79080370600008], [-122.48581537399991, 37.79070384600004], [-122.48557750799989, 37.790559847000054], [-122.4850531269999, 37.79036813300007], [-122.4842660519999, 37.789411709000035], [-122.4

In [347]:

# Assigning scores and updating the neighborhoods data
counter = 0
for neighborhood_name, distance in sorted_neighborhoods_min_distance:
    if distance >15*60:
        score = 3
    elif distance > 10*60:
        score = 2
    elif distance > 5*60:
        score = 1
    else:
        score = 0

    # Finding the corresponding feature and adding the walking_distance_to_bart property
    for feature in neighborhoods_data['features']:
        if feature['properties']['name'] == neighborhood_name:
            feature['properties']['driving_bart_score'] = score
            break

    counter += 1

print (neighborhoods_data["features"])

[{'type': 'Feature', 'properties': {'id': '0', 'link': 'http://en.wikipedia.org/wiki/Sea_Cliff,_San_Francisco,_California', 'name': 'Seacliff', 'nearest_park_distance': 0.0009406020243021834, 'park_score': 1, 'quad': 'NW', 'number_of_crimes': 70, 'crime_score': 3, 'bike_score': 2, 'driving_bart_score': 3}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-122.49345526799993, 37.78351817100008], [-122.49372649999992, 37.78724665100009], [-122.49358666699993, 37.78731259500006], [-122.49360569399994, 37.78752774600008], [-122.49283007399993, 37.787882585000034], [-122.4927566799999, 37.78773917700005], [-122.48982906399993, 37.789482184000065], [-122.48899105699991, 37.78928318700008], [-122.4878640209999, 37.78958817900008], [-122.48736904899994, 37.78942984100007], [-122.48598032899991, 37.79080370600008], [-122.48581537399991, 37.79070384600004], [-122.48557750799989, 37.790559847000054], [-122.4850531269999, 37.79036813300007], [-122.4842660519999, 37.789411709000035], [-122.4

In [348]:
# Saving the modified GeoJSON object
output_path = './neighborhoods_with_driving_bart_scores.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)



## Finding the same but for walking distance

In [353]:
with open('./neighborhoods_with_driving_bart_scores.geojson', 'r') as file:
    neighborhoods_data = json.load(file)

In [350]:
#function for getting minimum walking distance from a neighborhood to each station
def min_walking_distance_to_bart(neighborhood, stations):

    # Handling the multi-polygon structure
    neighborhood_coords = neighborhood['geometry']['coordinates'][0][0]
    neighborhood_polygon = Polygon(neighborhood_coords)
    centroid_coords = (neighborhood_polygon.centroid.y, neighborhood_polygon.centroid.x) # Swapping latitude and longitude
    
    min_distance = float('inf')
    for station_name, station_coords in stations.items():
        distance = get_route_summary(centroid_coords[0], centroid_coords[1], station_coords[0], station_coords[1], "Walking", api_key)
        if distance:
            min_distance = min(min_distance, distance)

    return min_distance


In [351]:
# Assuming neighborhoods_data and bart_stations are defined
neighborhoods_min_distance_dict = {}
for neighborhood in neighborhoods_data['features']:
    neighborhood_name = neighborhood['properties']['name']
    min_distance = min_walking_distance_to_bart(neighborhood, bart_stations)
    neighborhoods_min_distance_dict[neighborhood_name] = min_distance

print(neighborhoods_min_distance_dict)
# Sorting the distances
sorted_neighborhoods_min_distance = sorted(neighborhoods_min_distance_dict.items(), key=lambda x:x[1])




{'Seacliff': 5681, 'Lake Street': 4936, 'Presidio National Park': 4922, 'Presidio Terrace': 4045, 'Inner Richmond': 3871, 'Sutro Heights': 6436, 'Lincoln Park / Ft. Miley': 6410, 'Outer Richmond': 5291, 'Golden Gate Park': 4676, 'Presidio Heights': 3348, 'Laurel Heights / Jordan Park': 3037, 'Lone Mountain': 2720, 'Anza Vista': 2206, 'Cow Hollow': 3132, 'Union Street': 2388, 'Nob Hill': 1097, 'Marina': 3191, 'Telegraph Hill': 1186, 'Downtown / Union Square': 284, 'Tenderloin': 499, 'Civic Center': 393, 'Hayes Valley': 901, 'Alamo Square': 1410, 'Panhandle': 2092, 'Haight Ashbury': 2242, 'Lower Haight': 1283, 'Mint Hill': 855, 'Duboce Triangle': 1058, 'Cole Valley': 2203, 'Rincon Hill': 626, 'South Beach': 1089, 'South of Market': 705, 'Showplace Square': 1183, 'Mission Bay': 1673, 'Yerba Buena Island': 11380, 'Treasure Island': 12853, 'Mission Dolores': 518, 'Castro': 1094, 'Outer Sunset': 4935, 'Parkside': 3495, 'Stonestown': 2097, 'Parkmerced': 2348, 'Lakeshore': 4507, 'Golden Gate H

In [354]:

# Assigning scores and updating the neighborhoods data
counter = 0
for neighborhood_name, distance in sorted_neighborhoods_min_distance:
    if distance >15*60:
        score = 3
    elif distance > 10*60:
        score = 2
    elif distance > 5*60:
        score = 1
    else:
        score = 0

    # Finding the corresponding feature and adding the walking_distance_to_bart property
    for feature in neighborhoods_data['features']:
        if feature['properties']['name'] == neighborhood_name:
            feature['properties']['walking_bart_score'] = score
            break

    counter += 1

print (neighborhoods_data["features"])

[{'type': 'Feature', 'properties': {'id': '0', 'link': 'http://en.wikipedia.org/wiki/Sea_Cliff,_San_Francisco,_California', 'name': 'Seacliff', 'nearest_park_distance': 0.0009406020243021834, 'park_score': 1, 'quad': 'NW', 'number_of_crimes': 70, 'crime_score': 3, 'bike_score': 2, 'driving_bart_score': 3, 'walking_bart_score': 3}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-122.49345526799993, 37.78351817100008], [-122.49372649999992, 37.78724665100009], [-122.49358666699993, 37.78731259500006], [-122.49360569399994, 37.78752774600008], [-122.49283007399993, 37.787882585000034], [-122.4927566799999, 37.78773917700005], [-122.48982906399993, 37.789482184000065], [-122.48899105699991, 37.78928318700008], [-122.4878640209999, 37.78958817900008], [-122.48736904899994, 37.78942984100007], [-122.48598032899991, 37.79080370600008], [-122.48581537399991, 37.79070384600004], [-122.48557750799989, 37.790559847000054], [-122.4850531269999, 37.79036813300007], [-122.4842660519999, 37.

In [355]:
# Saving the modified GeoJSON object
output_path = './neighborhoods_with_walking_bart_scores.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)



# FINDING THE DISTANCE TO THE NEAREST WHOLE FOODS

In [359]:
#foursquare api key: fsq3i2LZ/jOJc5rTexXQdKYDi6SaCntsDl15fpCL0Yhct5k=


pp = pprint.PrettyPrinter(indent=1)


url = "https://api.foursquare.com/v3/places/search?ll=37.751858%2C-122.445943&radius=9000&chains=c356152c-f98c-4c43-8a1e-1d16cc9ea1c8&sort=DISTANCE"

headers = {
    "accept": "application/json",
}

response = requests.get(url, headers=headers)
res_object=eval(response.text)["results"]
whole_foods_data=[]
for result in res_object:
    whole_foods_data.append(result)
pp.pprint(whole_foods_data)
# Extracting necessary information from Whole Foods data
whole_foods_stores = [{
    'latitude': store['geocodes']['main']['latitude'],
    'longitude': store['geocodes']['main']['longitude']
} for store in whole_foods_data]
print (whole_foods_stores)
# Load neighborhoods data
with open('./neighborhoods_with_walking_bart_scores.geojson', 'r') as file:
    neighborhoods_data = json.load(file)

# Function to calculate distance
def calculate_distance(coords_1, coords_2):
    return geopy.distance.distance(coords_1, coords_2).meters

# Iterate through each neighborhood and find the distance to the nearest Whole Foods
for feature in neighborhoods_data['features']:
    neighborhood_coords = tuple(reversed(feature['geometry']['coordinates'][0][0][0]))
    distances = [calculate_distance(neighborhood_coords, (store['latitude'], store['longitude'])) for store in whole_foods_stores]
    min_distance = min(distances)
    feature['properties']['distance_from_whole_foods'] = min_distance


[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
                           'suffix': '.png'},
                  'id': 17069,
                  'name': 'Grocery Store'}],
  'chains': [{'id': 'c356152c-f98c-4c43-8a1e-1d16cc9ea1c8',
              'name': 'Whole Foods'}],
  'distance': 1324,
  'fsq_id': '4abd43f4f964a5204f8920e3',
  'geocodes': {'drop_off': {'latitude': 37.752285, 'longitude': -122.430991},
               'main': {'latitude': 37.751993, 'longitude': -122.430973},
               'roof': {'latitude': 37.751993, 'longitude': -122.430973}},
  'link': '/v3/places/4abd43f4f964a5204f8920e3',
  'location': {'address': '3950 24th St',
               'census_block': '060750211003004',
               'country': 'US',
               'cross_street': 'btwn Sanchez and Noe St',
               'dma': 'San Francisco-Oakland-San Jose',
               'formatted_address': '3950 24th St (btwn Sanchez and Noe St), '
                           

In [370]:
# Extracting necessary information from Whole Foods data
whole_foods_stores = [{
    'latitude': store['geocodes']['main']['latitude'],
    'longitude': store['geocodes']['main']['longitude']
} for store in whole_foods_data]
print (whole_foods_stores)
# Load neighborhoods data
with open('./neighborhoods_with_walking_bart_scores.geojson', 'r') as file:
    neighborhoods_data = json.load(file)

# Function to calculate distance
def calculate_distance(coords_1, coords_2):
    return geopy.distance.distance(coords_1, coords_2).meters

# Iterate through each neighborhood and find the distance to the nearest Whole Foods
for feature in neighborhoods_data['features']:
    neighborhood_coords = tuple(reversed(feature['geometry']['coordinates'][0][0][0]))
    distances = [calculate_distance(neighborhood_coords, (store['latitude'], store['longitude'])) for store in whole_foods_stores]
    min_distance = min(distances)
    feature['properties']['distance_from_whole_foods'] = min_distance


[{'latitude': 37.751993, 'longitude': -122.430973}, {'latitude': 37.769522, 'longitude': -122.452936}, {'latitude': 37.768616, 'longitude': -122.427227}, {'latitude': 37.723814, 'longitude': -122.454592}, {'latitude': 37.729053, 'longitude': -122.476175}, {'latitude': 37.764435, 'longitude': -122.402827}, {'latitude': 37.78998, 'longitude': -122.423402}, {'latitude': 37.781301, 'longitude': -122.399735}]


In [371]:
# Finding the quartiles
# Extract all distances to calculate quartiles
distances = [feature['properties']['distance_from_whole_foods'] for feature in neighborhoods_data['features']]

# Calculate quartiles
first_quartile = np.percentile(distances, 25)
second_quartile = np.percentile(distances, 50)
third_quartile = np.percentile(distances, 75)

# Assign scores based on quartiles
for feature in neighborhoods_data['features']:
    distance = feature['properties']['distance_from_whole_foods']
    
    if distance <= first_quartile:
        feature['properties']['whole_foods_score'] = 0
    elif distance <= second_quartile:
        feature['properties']['whole_foods_score'] = 1
    elif distance <= third_quartile:
        feature['properties']['whole_foods_score'] = 2
    else:
        feature['properties']['whole_foods_score'] = 3

# Saving the modified GeoJSON object
output_path = './neighborhoods_with_whole_foods_scores.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)


# Creating a file that stores whole foods coordinates as a geojson

In [372]:
# Constructing the GeoJSON structure
whole_foods_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for store in whole_foods_stores:
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [store['longitude'], store['latitude']]  # GeoJSON uses [lon, lat]
        },
        "properties": {}
    }
    whole_foods_geojson['features'].append(feature)

# Saving the modified GeoJSON object
output_path = './whole_foods_coordinates.geojson'
with open(output_path, 'w') as file:
    json.dump(whole_foods_geojson, file)


# FINDING THE DISTANCE TO THE NEAREST TRADER JOES

In [373]:

url = "https://api.foursquare.com/v3/places/search?ll=37.751858%2C-122.445943&radius=9000&chains=6adf56af-558f-4fc8-8bb0-bfb35dd622bc&sort=DISTANCE"
headers = {
    "accept": "application/json",
}

response = requests.get(url, headers=headers)
res_object=eval(response.text)["results"]
trader_joes_data=[]
for result in res_object:
    trader_joes_data.append(result)
pp.pprint(whole_foods_data)
# Extracting necessary information from Whole Foods data
trader_joes_stores = [{
    'latitude': store['geocodes']['main']['latitude'],
    'longitude': store['geocodes']['main']['longitude']
} for store in trader_joes_data]
print (trader_joes_stores)
# Load neighborhoods data
with open('./neighborhoods_with_whole_foods_scores.geojson', 'r') as file:
    neighborhoods_data = json.load(file)

# Function to calculate distance
def calculate_distance(coords_1, coords_2):
    return geopy.distance.distance(coords_1, coords_2).meters

# Iterate through each neighborhood and find the distance to the nearest Whole Foods
for feature in neighborhoods_data['features']:
    neighborhood_coords = tuple(reversed(feature['geometry']['coordinates'][0][0][0]))
    distances = [calculate_distance(neighborhood_coords, (store['latitude'], store['longitude'])) for store in whole_foods_stores]
    min_distance = min(distances)
    feature['properties']['distance_from_trader_joes'] = min_distance



[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
                           'suffix': '.png'},
                  'id': 17069,
                  'name': 'Grocery Store'}],
  'chains': [{'id': 'c356152c-f98c-4c43-8a1e-1d16cc9ea1c8',
              'name': 'Whole Foods'}],
  'distance': 1324,
  'fsq_id': '4abd43f4f964a5204f8920e3',
  'geocodes': {'drop_off': {'latitude': 37.752285, 'longitude': -122.430991},
               'main': {'latitude': 37.751993, 'longitude': -122.430973},
               'roof': {'latitude': 37.751993, 'longitude': -122.430973}},
  'link': '/v3/places/4abd43f4f964a5204f8920e3',
  'location': {'address': '3950 24th St',
               'census_block': '060750211003004',
               'country': 'US',
               'cross_street': 'btwn Sanchez and Noe St',
               'dma': 'San Francisco-Oakland-San Jose',
               'formatted_address': '3950 24th St (btwn Sanchez and Noe St), '
                           

In [374]:
# Extracting necessary information from Whole Foods data
trader_joes_stores = [{
    'latitude': store['geocodes']['main']['latitude'],
    'longitude': store['geocodes']['main']['longitude']
} for store in trader_joes_data]
print (trader_joes_stores)

# Function to calculate distance
def calculate_distance(coords_1, coords_2):
    return geopy.distance.distance(coords_1, coords_2).meters

# Iterate through each neighborhood and find the distance to the nearest Whole Foods
for feature in neighborhoods_data['features']:
    neighborhood_coords = tuple(reversed(feature['geometry']['coordinates'][0][0][0]))
    distances = [calculate_distance(neighborhood_coords, (store['latitude'], store['longitude'])) for store in whole_foods_stores]
    min_distance = min(distances)
    feature['properties']['distance_from_trader_joes'] = min_distance


[{'latitude': 37.783379, 'longitude': -122.447739}, {'latitude': 37.72669, 'longitude': -122.476321}, {'latitude': 37.770744, 'longitude': -122.407499}, {'latitude': 37.790654, 'longitude': -122.417993}, {'latitude': 37.785418, 'longitude': -122.405712}, {'latitude': 37.700019, 'longitude': -122.482322}, {'latitude': 37.805474, 'longitude': -122.41385}]


In [375]:
# Finding the quartiles


# Extract all distances to calculate quartiles
distances = [feature['properties']['distance_from_trader_joes'] for feature in neighborhoods_data['features']]

# Calculate quartiles
first_quartile = np.percentile(distances, 25)
second_quartile = np.percentile(distances, 50)
third_quartile = np.percentile(distances, 75)

# Assign scores based on quartiles
for feature in neighborhoods_data['features']:
    distance = feature['properties']['distance_from_trader_joes']
    
    if distance <= first_quartile:
        feature['properties']['trader_joes_score'] = 0
    elif distance <= second_quartile:
        feature['properties']['trader_joes_score'] = 1
    elif distance <= third_quartile:
        feature['properties']['trader_joes_score'] = 2
    else:
        feature['properties']['trader_joes_score'] = 3

# Saving the modified GeoJSON object
output_path = './neighborhoods_with_trader_joes_scores.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)


# File that stores trader joes coordinates

In [376]:
# Constructing the GeoJSON structure
trader_joes_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for store in trader_joes_stores:
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [store['longitude'], store['latitude']]  # GeoJSON uses [lon, lat]
        },
        "properties": {}
    }
    trader_joes_geojson['features'].append(feature)

# Saving the modified GeoJSON object
output_path = './trader_joes_coordinates.geojson'
with open(output_path, 'w') as file:
    json.dump(trader_joes_geojson, file)


# FIND THE DISTANCE TO THE NEAREST GYM

In [377]:
url = "https://api.foursquare.com/v3/places/search?query=gym&ll=37.751858%2C-122.445943&radius=12000&sort=DISTANCE"

headers = {
    "accept": "application/json",
}

response = requests.get(url, headers=headers)
gym_response=eval(response.text)["results"]

gym_data=[]
for result in gym_response:
    gym_data.append(result)
pp.pprint(gym_data)
# Extracting necessary information from Gym data
gyms = [{
    'name': gym["name"],
    'latitude': gym['geocodes']['main']['latitude'],
    'longitude': gym['geocodes']['main']['longitude']
} for gym in gym_data]
print (gyms)
# Load neighborhoods data
with open('./neighborhoods_with_trader_joes_scores.geojson', 'r') as file:
    neighborhoods_data = json.load(file)

# Iterate through each neighborhood and find the distance to the nearest Whole Foods
for feature in neighborhoods_data['features']:
    neighborhood_coords = tuple(reversed(feature['geometry']['coordinates'][0][0][0]))
    distances = [calculate_distance(neighborhood_coords, (gym['latitude'], gym['longitude'])) for gym in gyms]
    min_distance = min(distances)
    feature['properties']['distance_from_gym'] = min_distance



[{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
                           'suffix': '.png'},
                  'id': 18021,
                  'name': 'Gym and Studio'}],
  'chains': [],
  'distance': 729,
  'fsq_id': '7ce0dba15b7179147b7100b1',
  'geocodes': {'drop_off': {'latitude': 37.746003, 'longitude': -122.442278},
               'main': {'latitude': 37.746046, 'longitude': -122.442085},
               'roof': {'latitude': 37.746046, 'longitude': -122.442085}},
  'link': '/v3/places/7ce0dba15b7179147b7100b1',
  'location': {'address': '115 Red Rock Way',
               'census_block': '060750216002003',
               'country': 'US',
               'dma': 'San Francisco-Oakland-San Jose',
               'formatted_address': '115 Red Rock Way, San Francisco, CA 94131',
               'locality': 'San Francisco',
               'postcode': '94131',
               'region': 'CA'},
  'name': 'Dhv Gym',
  'related_places': {}},
 {'categor

In [378]:

# Extract all distances to calculate quartiles
distances = [feature['properties']['distance_from_gym'] for feature in neighborhoods_data['features']]

# Calculate quartiles
first_quartile = np.percentile(distances, 25)
second_quartile = np.percentile(distances, 50)
third_quartile = np.percentile(distances, 75)

# Assign scores based on quartiles
for feature in neighborhoods_data['features']:
    distance = feature['properties']['distance_from_gym']
    
    if distance <= first_quartile:
        feature['properties']['gym_score'] = 0
    elif distance <= second_quartile:
        feature['properties']['gym_score'] = 1
    elif distance <= third_quartile:
        feature['properties']['gym_score'] = 2
    else:
        feature['properties']['gym_score'] = 3

# Saving the modified GeoJSON object
output_path = './neighborhoods_with_gym_scores.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)

## STORE GYM COORDINATES AND NAMES

In [379]:
# Constructing the GeoJSON structure
gym_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for gym in gyms:
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [gym['longitude'], gym['latitude']],  # GeoJSON uses [lon, lat],
            
        },
        "properties": {
            "name": gym["name"]
        }
    }
    gym_geojson['features'].append(feature)

# Saving the modified GeoJSON object
output_path = './gym_coordinates.geojson'
with open(output_path, 'w') as file:
    json.dump(gym_geojson, file)


# FINDING RENT PRICES BY ZIP CODE

## Assign each neighborhood a zip code like we did with quadrants

In [5]:
# Function to convert GeoJSON to GeoDataFrame
def geojson_to_geodataframe(geojson):
    features = geojson['features']
    geometries = [shape(feature['geometry']) for feature in features]
    properties = [feature['properties'] for feature in features]
    return GeoDataFrame(properties, geometry=geometries)

# Loading neighborhoods and zip GeoJSON files
with open('./neighborhoods_with_gym_scores.geojson', 'r') as file:
    neighborhoods_geojson = json.load(file)
with open('./zip_codes.geojson', 'r') as file:
    zip_geojson = json.load(file)

# Converting to GeoDataFrames
neighborhoods_gdf = geojson_to_geodataframe(neighborhoods_geojson)
zip_gdf = geojson_to_geodataframe(zip_geojson)

# Performing spatial join to find the overlapping quadrants for each neighborhood
joined_gdf = GeoDataFrame(neighborhoods_gdf)
joined_gdf['zip'] = None  # Initialize the 'quad' column

for idx, neighborhood in neighborhoods_gdf.iterrows():
    for _, code in zip_gdf.iterrows():
        if neighborhood['geometry'].intersects(code['geometry']):
            joined_gdf.at[idx, 'zip'] = code['zip']
            break

# Converting back to GeoJSON
joined_geojson = json.loads(joined_gdf.to_json())

# Saving the updated GeoJSON file with the "quad" property added
updated_file_path = './neighborhoods_with_zips.geojson'
with open(updated_file_path, 'w') as file:
    json.dump(joined_geojson, file)

updated_file_path

'./neighborhoods_with_zips.geojson'

In [29]:
# Check to see if zips were updated and how many zips there are total
with open('./neighborhoods_with_zips.geojson', 'r') as file:
    neighborhoods_data = json.load(file)
zip_set=set()
for neighborhood in neighborhoods_data["features"]:
    zip_set.add(neighborhood["properties"]["zip"])


In [26]:
#Using zip set, we want to make api calls and store the average cost per zip_code of each neighborhood

cost_zip_map={}
for code in zip_set:
    url = "https://api.rentcast.io/v1/markets?zipCode="+code+"&historyRange=6"

    headers = {
        "accept": "application/json",
        "X-Api-Key": rentcast_key
    }

    response = requests.get(url, headers=headers)
    cost_zip_map[code]=(eval(response.text)["rentalData"]["averageRent"])

cost_zip_map

{'94131': 4208,
 '94133': 4060,
 '94105': 4944,
 '94109': 3323,
 '94104': 6321,
 '94134': 3419,
 '94116': 3961,
 '94115': 3980,
 '94123': 3970,
 '94132': 3523,
 '94127': 4617,
 '94110': 4111,
 '94121': 3401,
 '94130': 3423,
 '94158': 4531,
 '94114': 4156,
 '94103': 3567,
 '94117': 3946,
 '94102': 2955,
 '94124': 3656,
 '94112': 3605,
 '94118': 3911}

In [30]:
#Using our new array, we want to create a new table in the column called 'rental cost' and add whichever neighborhoods belong
for feature in neighborhoods_data['features']:
    code= feature['properties']['zip']
    feature['properties']['cost'] = cost_zip_map[code]

# Saving the modified GeoJSON object
output_path = './neighborhoods_with_costs.geojson'
with open(output_path, 'w') as file:
    json.dump(neighborhoods_data, file)
